![Buffalo Soccer Header](https://carload.com/g/BuffaloSoccerWide400.jpg)

# Part 2, Prepare the Master File

Now that we have created a dataframe for all the midweek games, it's time to turn to the meat of the project. I previously downloaded the match information for three Bundesliga seasons from Football-Data (https://www.football-data.co.uk/). We'll start by merging them into one dataframe.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx

In [3]:
df2021 = pd.read_csv('data/buffalo/D1_2021.csv')
df2022 = pd.read_csv('data/buffalo/D1_2022.csv')
df2023 = pd.read_csv('data/buffalo/D1_2023.csv')

In [4]:
df2021.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,D1,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,3.01,0.75,1.99,1.94,1.99,1.93,2.10,1.96,1.98,1.88
1,D1,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,2.30,0.25,1.97,1.96,1.98,1.94,2.02,1.99,1.96,1.91
2,D1,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,1.64,0.25,1.75,2.05,1.79,2.16,1.80,2.20,1.76,2.12
3,D1,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,2.16,-0.75,2.02,1.77,2.09,1.83,2.12,1.86,2.07,1.80
4,D1,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,1.76,0.00,1.92,2.01,1.92,2.01,1.95,2.02,1.91,1.96


In [5]:
dfAll = pd.concat([df2021, df2022, df2023])
print(dfAll.shape)

(918, 105)


There are _a_ _lot_ of fields in this data that relate to different odds offered at different times by different sports books. (See the BuFFALO_Notes.txt file in this GitHub folder for all of the column meanings.) We don't want to drill down to compare them, so we'll just keep the average odds for each result.

In [6]:
# Drop almost all of the dozens(!) of odds fields
dfActive = dfAll.iloc[:, 0:23]
# Then retrieve just the average odds for home win, draw, and away win, just in case
dfActive['AvgH'] = dfAll['AvgH']
dfActive['AvgD'] = dfAll['AvgD']
dfActive['AvgA'] = dfAll['AvgA']
dfActive.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,D1,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,4,3,8,2,0,0,0,4.54,4.32,1.70
1,D1,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,9,6,5,1,2,0,0,3.19,3.68,2.17
2,D1,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,10,4,9,3,4,0,0,2.92,3.27,2.52
3,D1,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,8,9,2,0,1,0,0,1.86,3.90,3.98
4,D1,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,5,1,6,0,1,0,0,2.79,3.36,2.57


We'll need a target field for the learning model, and that Div column looks like the perfect spot for it.

Note that in most soccer leagues (including the Bundesliga), the winning team gets 3 points in the standings for a win, both teams get 1 standings point for a draw, and of course 0 for a loss. This is to encourage playing for wins; a team with one win and one loss will rank ahead of a team with two draws.

In [8]:
# We don't need the Div field, since all matches are D1
# But it's in the perfect position to put our 'y' result field
# 3 for a home win, 1 for a draw, 0 for an away win
dfActive.rename(columns = {"Div": "y"}, inplace = True)
dfActive['y'] = np.where(dfActive['FTR'] == 'H', 3, 0)
dfActive.head()

,y,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,0,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,4,3,8,2,0,0,0,4.54,4.32,1.70
1,0,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,9,6,5,1,2,0,0,3.19,3.68,2.17
2,0,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,10,4,9,3,4,0,0,2.92,3.27,2.52
3,3,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,8,9,2,0,1,0,0,1.86,3.90,3.98
4,0,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,5,1,6,0,1,0,0,2.79,3.36,2.57


In [9]:
# y is now 3 or 0, but we want to change the 0 to 1 if it was a draw
dfActive['y'] = np.where(dfActive['FTR'] == 'D', 1, dfActive['y'])
dfActive.head()

,y,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,1,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,4,3,8,2,0,0,0,4.54,4.32,1.70
1,0,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,9,6,5,1,2,0,0,3.19,3.68,2.17
2,1,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,10,4,9,3,4,0,0,2.92,3.27,2.52
3,3,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,8,9,2,0,1,0,0,1.86,3.90,3.98
4,1,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,5,1,6,0,1,0,0,2.79,3.36,2.57


### Create OrdDates

How can we easily tell whether a team has played a match in a midweek league over the previous six days? The solution is to convert the Day/Month/Year string into ordinal dates, which we also did for the OpenLiga data in another notebook in this project.

In [10]:
from datetime import datetime
# We want to create an OrdDate field from the Date string for easy date comparisons

# I hate jamming several processes into one line, but it's the best way to go in a Pandas dataframe
# In a new column, convert the 'Date' string to a datetime object, then convert _that_ to its ordinal
dfActive['OrdDate'] = pd.to_datetime(dfActive['Date'], format='%d/%m/%Y').apply(lambda x: x.toordinal())
dfActive.head()

,y,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA,OrdDate
0,1,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,1,...,3,8,2,0,0,0,4.54,4.32,1.70,738015
1,0,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,1,...,6,5,1,2,0,0,3.19,3.68,2.17,738016
2,1,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,0,...,4,9,3,4,0,0,2.92,3.27,2.52,738016
3,3,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,0,...,9,2,0,1,0,0,1.86,3.90,3.98,738016
4,1,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,1,...,1,6,0,1,0,0,2.79,3.36,2.57,738016


In [11]:
# Now move the OrdDate column next to the Date for easier sanity checks
ordCol = dfActive.pop('OrdDate')
dfActive.insert(1, 'OrdDate', ordCol)
dfActive.head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,1,738015,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,...,4,3,8,2,0,0,0,4.54,4.32,1.70
1,0,738016,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,...,9,6,5,1,2,0,0,3.19,3.68,2.17
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,10,4,9,3,4,0,0,2.92,3.27,2.52
3,3,738016,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,...,8,9,2,0,1,0,0,1.86,3.90,3.98
4,1,738016,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,...,5,1,6,0,1,0,0,2.79,3.36,2.57


Add some columns in preparation for merging midweek data.

In [12]:
# Add columns for a possible mid-week match just before this one
#  MH - today's HomeTeam,  MWA - today's AwayTeam
# MHOrd (today's HomeTeam, 0 if no game)
# MAOrd (today's AwayTeam, 0 if no game)
# MHLoc (today's HomeTeam, 1 if home, 0 if away)
# MALoc (today's AwayTeam, 1 if home, 0 if away)
# MHRes (regardless of home/away, 3 if won, 1 if draw, 0 if loss)
# MARes (regardless of home/away, 3 if won, 1 if draw, 0 if loss)
dfActive['MHOrd'] = 0
dfActive['MAOrd'] = 0
dfActive['MHLoc'] = 0
dfActive['MALoc'] = 0
dfActive['MHRes'] = 0
dfActive['MARes'] = 0
dfActive.head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
0,1,738015,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,...,0,4.54,4.32,1.70,0,0,0,0,0,0
1,0,738016,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,...,0,3.19,3.68,2.17,0,0,0,0,0,0
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,0,2.92,3.27,2.52,0,0,0,0,0,0
3,3,738016,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,...,0,1.86,3.90,3.98,0,0,0,0,0,0
4,1,738016,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,...,0,2.79,3.36,2.57,0,0,0,0,0,0


### Build Team List

The list of all the teams in the dataframe will become very important as we create dictionaries of each team's week-by-week results.

In [15]:
teamList = dfActive['HomeTeam'].unique()
teamList.sort()
print(teamList)
print(type(teamList))

['Augsburg' 'Bayern Munich' 'Bielefeld' 'Bochum' 'Darmstadt' 'Dortmund'
 'Ein Frankfurt' 'FC Koln' 'Freiburg' 'Greuther Furth' 'Heidenheim'
 'Hertha' 'Hoffenheim' 'Leverkusen' "M'gladbach" 'Mainz' 'RB Leipzig'
 'Schalke 04' 'Stuttgart' 'Union Berlin' 'Werder Bremen' 'Wolfsburg']
<class 'numpy.ndarray'>


In [16]:
# Let's build a dictionary of team-based records
dfDict = {}
for eachTeam in teamList:
    dfTempHome = dfActive[dfActive['HomeTeam'] == eachTeam]
    dfTempAway = dfActive[dfActive['AwayTeam'] == eachTeam]
    dfTemp = pd.concat([dfTempHome, dfTempAway])
    dfTemp.sort_values(by=['OrdDate'], inplace = True)
    dfDict[eachTeam] = dfTemp

Here's an example of one team-based dataframe, sorted by date.

In [23]:
dfDict['Freiburg'].head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,0,2.92,3.27,2.52,0,0,0,0,0,0
12,3,738023,21/08/2021,14:30,Freiburg,Dortmund,2,1,H,1,...,0,5.50,4.59,1.55,0,0,0,0,0,0
23,0,738030,28/08/2021,14:30,Stuttgart,Freiburg,2,3,A,2,...,0,2.23,3.53,3.22,0,0,0,0,0,0
27,1,738044,11/09/2021,14:30,Freiburg,FC Koln,1,1,D,0,...,1,2.14,3.70,3.25,0,0,0,0,0,0
40,1,738051,18/09/2021,14:30,Mainz,Freiburg,0,0,D,0,...,0,2.07,3.58,3.53,0,0,0,0,0,0


## Prepare OpenLiga Data for Merging

Now we'll pull in the OpenLiga data and rename its columns to match the master dataframe.

In [32]:
from pandas import read_pickle
dfOpenLiga = pd.read_pickle('data/buffalo/openliga.pkl')
dfOpenLiga.head()

,matchID,ordDate,team1Name,team2Name,result
0,62820,738133,Sparta,Brondby,3
1,62821,738133,Lyon,Rangers,1
2,62822,738133,,,1
3,62823,738133,,PSV,3
4,62824,738133,,mos spartak,0


In [42]:
dfOpenLiga.rename(columns={"team1Name": "HomeTeam", 
                           "team2Name" : "AwayTeam",
                           "ordDate" : "OrdDate"}, inplace=True)
dfOpenLiga.head()

,matchID,OrdDate,HomeTeam,AwayTeam,result
0,62820,738133,Sparta,Brondby,3
1,62821,738133,Lyon,Rangers,1
2,62822,738133,,,1
3,62823,738133,,PSV,3
4,62824,738133,,mos spartak,0


Let's make sure that we get the results we want in merging home and away games for a given team by trying it on one in particular.

In [28]:
dfBayHome = dfOpenLiga[dfOpenLiga['team1Name'] == 'Bayern Munich']
dfBayAway = dfOpenLiga[dfOpenLiga['team2Name'] == 'Bayern Munich']
dfAllBay = pd.concat([dfBayHome, dfBayAway]).sort_values('ordDate', ascending=True)
dfAllBay.head()

,matchID,ordDate,team1Name,team2Name,result
31,61596,738027,,Bayern Munich,0
2,62580,738047,Barcelona,Bayern Munich,0
26,62604,738062,Bayern Munich,Kiew,3
42,62620,738083,Benfica,Bayern Munich,0
45,62552,738090,M'gladbach,Bayern Munich,3


That looks good, so we'll do it for all of the teams. Then we'll check an example.

In [44]:
# Let's build a dictionary of team-based records FOR OPENLIGA
dfDictOL = {}
for eachTeam in teamList:
    dfTempHome = dfOpenLiga[dfOpenLiga['HomeTeam'] == eachTeam]
    dfTempAway = dfOpenLiga[dfOpenLiga['AwayTeam'] == eachTeam]
    dfTemp = pd.concat([dfTempHome, dfTempAway])
    dfTemp.sort_values(by=['OrdDate'], inplace = True)
    dfDictOL[eachTeam] = dfTemp


In [45]:
dfDictOL['Augsburg'].head()

,matchID,OrdDate,HomeTeam,AwayTeam,result
6,61611,738009,Greifswald,Augsburg,0
41,62551,738090,Bochum,Augsburg,3
22,63545,738367,,Augsburg,0
44,64788,738447,Augsburg,Bayern Munich,0
20,66605,738745,Unterhaching,Augsburg,3


Here is the function that provides the midweek information we care about. Given a team and a date, did that team play within the previous six days? If so, was it at home or on the road? Did the team win, lose, or draw?

In [159]:
def checkMidWeek(orddate, team):
    mDay = orddate
    m6 = mDay - 6
    mwYes = 0
    mwHome = 0
    mwResult = 0
    midWeek = dfDictOL[team][(m6 <= dfDictOL[team]['OrdDate']) & (dfDictOL[team]['OrdDate'] < mDay)]
    if midWeek.shape[0]:
        mwYes = 1
        if midWeek.HomeTeam.iat[0] == team:
            mwHome = 1
        mwResult = midWeek.result.iat[0]
        # Return the result for the team
        #   That is, if visiting, flip 0 to 3 and vice versa
        if 0 == mwHome:
            if 3 == mwResult:
                mwResult = 0
            elif 0 == mwResult:
                mwResult = 3

    return mwYes, mwHome, mwResult

In [ ]:
# Setting some date constants
# We want Matchday 4-30 of each season for training
# and Matchday 31-34 for validation
# Constants represent first day of Matchday 4 & 31,
#  last day of Matchday 30. Only day of Matchday 34.

M4_2021  = 738044 # Sept. 11, 2021
M30_2021 = 738262 # April 17, 2022
M31_2021 = 738267 # April 22, 2022
M34_2021 = 738289 # May 14, 2022

M4_2022  = 738393 # Aug. 26, 2022
M30_2022 = 738640 # April 30, 2023
M31_2022 = 738645 # May 5, 2023
M34_2022 = 738667 # May 27, 2023

M4_2023  = 738778 # Sept. 15, 2023
M30_2023 = 738997 # April 21, 2024
M31_2023 = 739002 # April 26, 2024
M34_2023 = 739024 # May 18, 2024

In [75]:
dfDict['Freiburg'].head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,0,2.92,3.27,2.52,0,0,0,0,0,0
12,3,738023,21/08/2021,14:30,Freiburg,Dortmund,2,1,H,1,...,0,5.50,4.59,1.55,0,0,0,0,0,0
23,0,738030,28/08/2021,14:30,Stuttgart,Freiburg,2,3,A,2,...,0,2.23,3.53,3.22,0,0,0,0,0,0
27,1,738044,11/09/2021,14:30,Freiburg,FC Koln,1,1,D,0,...,1,2.14,3.70,3.25,0,0,0,0,0,0
40,1,738051,18/09/2021,14:30,Mainz,Freiburg,0,0,D,0,...,0,2.07,3.58,3.53,0,0,0,0,0,0


In [90]:
back3 = pd.Series()
back2 = pd.Series()
back1 = pd.Series()
numRecords = len(dfDict['Freiburg'])
for eachRecord in range(numRecords):
    back3 = back2
    back2 = back1
    back1 = dfDict['Freiburg'].iloc[eachRecord]

dfDict['Freiburg'].tail()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
269,1,738997,21/04/2024,18:30,Freiburg,Mainz,1,1,D,1,...,0,2.58,3.56,2.67,0,0,0,0,0,0
273,0,739003,27/04/2024,14:30,Freiburg,Wolfsburg,1,2,A,1,...,0,2.33,3.56,3.00,0,0,0,0,0,0
284,1,739010,04/05/2024,17:30,FC Koln,Freiburg,0,0,D,0,...,0,2.36,3.64,2.90,0,0,0,0,0,0
290,1,739017,11/05/2024,14:30,Freiburg,Heidenheim,1,1,D,1,...,0,1.73,4.06,4.59,0,0,0,0,0,0
303,3,739024,18/05/2024,14:30,Union Berlin,Freiburg,2,1,H,0,...,0,2.13,3.47,3.51,0,0,0,0,0,0


In [100]:
dfDict['Freiburg'].MHOrd = 0
dfDict['Freiburg'].tail()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
269,1,738997,21/04/2024,18:30,Freiburg,Mainz,1,1,D,1,...,0,2.58,3.56,2.67,0,0,0,0,0,0
273,0,739003,27/04/2024,14:30,Freiburg,Wolfsburg,1,2,A,1,...,0,2.33,3.56,3.00,0,0,0,0,0,0
284,1,739010,04/05/2024,17:30,FC Koln,Freiburg,0,0,D,0,...,0,2.36,3.64,2.90,0,0,0,0,0,0
290,1,739017,11/05/2024,14:30,Freiburg,Heidenheim,1,1,D,1,...,0,1.73,4.06,4.59,0,0,0,0,0,0
303,3,739024,18/05/2024,14:30,Union Berlin,Freiburg,2,1,H,0,...,0,2.13,3.47,3.51,0,0,0,0,0,0


In [150]:
print(dfActive.iloc[0])

y                       1
OrdDate            738015
Date           13/08/2021
Time                19:30
HomeTeam       M'gladbach
AwayTeam    Bayern Munich
FTHG                    1
FTAG                    1
FTR                     D
HTHG                    1
HTAG                    1
HTR                     D
HS                     11
AS                     20
HST                     5
AST                    10
HF                      7
AF                      4
HC                      3
AC                      8
HY                      2
AY                      0
HR                      0
AR                      0
AvgH                 4.54
AvgD                 4.32
AvgA                  1.7
MHOrd                   0
MAOrd                   0
MHLoc                   0
MALoc                   0
MHRes                   0
MARes                   0
Name: 0, dtype: object


In [153]:
print(dfActive.iloc[0].HomeTeam)
print(type(dfActive.iloc[0].HomeTeam))

M'gladbach
<class 'str'>


## Build the Midweek Dataframe

After trying several paths (not shown here) to get the midweek data into primary dataframe, I determined that the best and surest way to proceed was to build a separate dataframe from scratch, then merge it like a database table. For each match in the primary dataframe, we check for midweek information and add it to a smaller record.

In turn, the best way to build a dataframe while iterating through another structure is to build a list of lists, then add an array of column names.

In [161]:
# Build a list of lists to use with column names for Midweek Summary df
mwSumList = list()
for index, row in dfActive.iterrows():
    rowList = list()
    homeString = row["HomeTeam"]
    awayString = row["AwayTeam"]
    ordNumber = row["OrdDate"]
    rowList.append(ordNumber)
    rowList.append(homeString)
    rowList.append(awayString)
    mwYesH, mwHomeH, mwResultH = checkMidWeek(ordNumber, homeString)
    mwYesA, mwHomeA, mwResultA = checkMidWeek(ordNumber, awayString)
    rowList.append(mwYesH)
    rowList.append(mwYesA)
    rowList.append(mwHomeH)
    rowList.append(mwHomeA)
    rowList.append(mwResultH)
    rowList.append(mwResultA)
    mwSumList.append(rowList)


In [163]:
dfMidSummary = pd.DataFrame(mwSumList, columns = ['OrdDate', 'HomeTeam', 'AwayTeam', 
                                                  'MHOrd', 'MAOrd', 'MHLoc', 
                                                  'MALoc', 'MHRes', 'MARes'])
dfMidSummary.head()

,OrdDate,HomeTeam,AwayTeam,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
0,738015,M'gladbach,Bayern Munich,1,0,0,0,3,0
1,738016,Augsburg,Hoffenheim,0,1,0,0,0,3
2,738016,Bielefeld,Freiburg,0,1,0,0,0,3
3,738016,Stuttgart,Greuther Furth,0,0,0,0,0,0
4,738016,Union Berlin,Leverkusen,1,0,0,0,3,0


In [164]:
print(dfActive.shape)
print(dfMidSummary.shape)

(918, 33)
(918, 9)


Good! We have the same number of midweek summaries as primary matches. Now we'll drop those earlier, still empty headings from the primary dataframe, then merge in the more populated columns from the midweek dataframe. The result is a joined dataframe.

In [165]:
dfActive.drop(columns = ['MHOrd', 'MAOrd', 'MHLoc', 'MALoc', 'MHRes', 'MARes'], inplace = True)

In [166]:
dfActive.head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,1,738015,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,...,4,3,8,2,0,0,0,4.54,4.32,1.70
1,0,738016,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,...,9,6,5,1,2,0,0,3.19,3.68,2.17
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,10,4,9,3,4,0,0,2.92,3.27,2.52
3,3,738016,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,...,8,9,2,0,1,0,0,1.86,3.90,3.98
4,1,738016,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,...,5,1,6,0,1,0,0,2.79,3.36,2.57


In [169]:
dfJoined = pd.merge(dfActive, dfMidSummary, on=['OrdDate', 'HomeTeam', 'AwayTeam'])

In [170]:
dfJoined.head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
0,1,738015,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,...,0,4.54,4.32,1.70,1,0,0,0,3,0
1,0,738016,14/08/2021,14:30,Augsburg,Hoffenheim,0,4,A,0,...,0,3.19,3.68,2.17,0,1,0,0,0,3
2,1,738016,14/08/2021,14:30,Bielefeld,Freiburg,0,0,D,0,...,0,2.92,3.27,2.52,0,1,0,0,0,3
3,3,738016,14/08/2021,14:30,Stuttgart,Greuther Furth,5,1,H,2,...,0,1.86,3.90,3.98,0,0,0,0,0,0
4,1,738016,14/08/2021,14:30,Union Berlin,Leverkusen,1,1,D,1,...,0,2.79,3.36,2.57,1,0,0,0,3,0


In [172]:
dfJoined.to_pickle('data/buffalo/joined.pkl')

Once again, we'll build a dictionary of records by team, now with meaningful midweek information.

In [173]:
# Let's build a dictionary of team-based records
dfDict = {}
for eachTeam in teamList:
    dfTempHome = dfJoined[dfJoined['HomeTeam'] == eachTeam]
    dfTempAway = dfJoined[dfJoined['AwayTeam'] == eachTeam]
    dfTemp = pd.concat([dfTempHome, dfTempAway])
    dfTemp.sort_values(by=['OrdDate'], inplace = True)
    dfDict[eachTeam] = dfTemp

In [174]:
dfDict['Bayern Munich'].head()

,y,OrdDate,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AR,AvgH,AvgD,AvgA,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes
0,1,738015,13/08/2021,19:30,M'gladbach,Bayern Munich,1,1,D,1,...,0,4.54,4.32,1.70,1,0,0,0,3,0
17,3,738024,22/08/2021,16:30,Bayern Munich,FC Koln,3,2,H,0,...,0,1.13,9.35,19.76,0,0,0,0,0,0
24,3,738030,28/08/2021,17:30,Bayern Munich,Hertha,5,0,H,2,...,0,1.18,7.96,14.67,1,0,0,0,3,0
32,0,738044,11/09/2021,17:30,RB Leipzig,Bayern Munich,1,4,A,0,...,0,2.99,3.87,2.22,0,0,0,0,0,0
38,3,738051,18/09/2021,14:30,Bayern Munich,Bochum,7,0,H,4,...,0,1.07,13.19,31.99,1,0,0,0,3,0


## Build the Three-Week-Back Record Set

We have reached one of the final steps in the data wrangling process. What's the best way to put three weeks of match data for two teams into one flat record for a data learning model to digest? 

For this step, we'll create a list of single-match results for each team. Each week will have all of the relevant information for the match, and starting in the fourth week, we'll also include the midweek information.

In [218]:
# Now we get to build a dict, with a list of result lists for each team
resultDict = {}
for eachTeam in teamList:
    teamDict = {}
    back3 = list()
    back2 = list()
    back1 = list()
    for index, row in dfDict[eachTeam].iterrows():
        if len(back3) > 0:
            # This must be 4th week or later, save full three weeks back
            resultList = list()
            todayList = list()
            homeString = row["HomeTeam"]
            awayString = row["AwayTeam"]
            ordNumber = row["OrdDate"]
            todayList.append(row['y'])
            todayList.append(ordNumber)
            todayList.append(homeString)
            todayList.append(awayString)
            todayList.append(row['MHOrd'])
            todayList.append(row['MAOrd'])
            todayList.append(row['MHLoc'])
            todayList.append(row['MALoc'])
            todayList.append(row['MHRes'])
            todayList.append(row['MARes'])
            todayList.append(row['AvgH'])
            todayList.append(row['AvgD'])
            todayList.append(row['AvgA'])
            resultList.append(todayList)
            resultList.append(back1)
            resultList.append(back2)
            resultList.append(back3)
            teamDict[ordNumber] = resultList
        
        # Now that the resultList is saved, we'll shift the other lists
        back3 = back2
        back2 = back1
        #reset back1 and fill it
        back1 = list()
        homeString = row["HomeTeam"]
        awayString = row["AwayTeam"]
        ordNumber = row["OrdDate"]
        back1.append(row['y'])
        back1.append(ordNumber)
        back1.append(homeString)
        back1.append(awayString)
        back1.append(row['FTHG'])
        back1.append(row['FTAG'])
        back1.append(row['FTR'])
        back1.append(row['HTHG'])
        back1.append(row['HTAG'])
        back1.append(row['HTR'])
        back1.append(row['HS'])
        back1.append(row['AS'])
        back1.append(row['HST'])
        back1.append(row['AST'])
        back1.append(row['HF'])
        back1.append(row['AF'])
        back1.append(row['HC'])
        back1.append(row['AC'])
        back1.append(row['HY'])
        back1.append(row['AY'])
        back1.append(row['HR'])
        back1.append(row['AR'])
        back1.append(row['AvgH'])
        back1.append(row['AvgD'])
        back1.append(row['AvgA'])
        
    resultDict[eachTeam] = teamDict
    

In [216]:
print(resultDict["M'gladbach"])

{738045: [[3, 738045, "M'gladbach", 'Bielefeld', 0, 0, 0, 0, 0, 0, 1.46, 4.68, 6.86], [3, 738031, 'Union Berlin', "M'gladbach", 2, 1, 'H', 2, 0, 'H', 12, 18, 4, 5, 15, 6, 4, 8, 2, 0, 0, 0, 2.58, 3.3, 2.82], [3, 738023, 'Leverkusen', "M'gladbach", 4, 0, 'H', 2, 0, 'H', 13, 11, 6, 6, 9, 11, 7, 3, 4, 2, 0, 0, 2.32, 3.62, 2.95], [1, 738015, "M'gladbach", 'Bayern Munich', 1, 1, 'D', 1, 1, 'D', 11, 20, 5, 10, 7, 4, 3, 8, 2, 0, 0, 0, 4.54, 4.32, 1.7]], 738051: [[3, 738051, 'Augsburg', "M'gladbach", 0, 0, 0, 0, 0, 0, 4.1, 3.9, 1.84], [3, 738045, "M'gladbach", 'Bielefeld', 3, 1, 'H', 1, 1, 'D', 17, 18, 4, 7, 10, 18, 3, 4, 1, 1, 0, 0, 1.46, 4.68, 6.86], [3, 738031, 'Union Berlin', "M'gladbach", 2, 1, 'H', 2, 0, 'H', 12, 18, 4, 5, 15, 6, 4, 8, 2, 0, 0, 0, 2.58, 3.3, 2.82], [3, 738023, 'Leverkusen', "M'gladbach", 4, 0, 'H', 2, 0, 'H', 13, 11, 6, 6, 9, 11, 7, 3, 4, 2, 0, 0, 2.32, 3.62, 2.95]], 738058: [[3, 738058, "M'gladbach", 'Dortmund', 0, 0, 0, 0, 0, 0, 4.09, 4.26, 1.77], [3, 738051, 'Augsburg'

Let's give this a quick sanity check. In the absence of matchIDs, we will identify each game by its date and home team.

In [186]:
# Test loading one record
dfOne = dfJoined.loc[(dfJoined['HomeTeam'] == 'Bayern Munich') & (dfJoined['OrdDate'] == 738051)]
print(dfOne)

    y  OrdDate        Date   Time       HomeTeam AwayTeam  FTHG  FTAG FTR  \
38  3   738051  18/09/2021  14:30  Bayern Munich   Bochum     7     0   H   

    HTHG  ...  AR  AvgH   AvgD   AvgA  MHOrd  MAOrd  MHLoc  MALoc  MHRes  \
38     4  ...   0  1.07  13.19  31.99      1      0      0      0      3   

    MARes  
38      0  

[1 rows x 33 columns]


Here is the two lists of lists for the match selected above. Note that both the home and away team lists start with the identical, shorter list of information about that game. Combining them into one long, flat list, minus the redundant information, is simple.

In [200]:
homeList = resultDict['Bayern Munich'].get(738051)
print(homeList)
awayString = dfOne['AwayTeam']
for el in awayString:
    continue
awayList = resultDict[el].get(738051)
print(awayList)

[[3, 738051, 'Bayern Munich', 'Bochum', 1, 0, 0, 0, 3, 0, 1.07, 13.19, 31.99], [0, 738044, 'RB Leipzig', 'Bayern Munich', 1, 4, 'A', 0, 1, 'A', 10, 18, 4, 6, 9, 13, 2, 6, 0, 0, 0, 0, 2.99, 3.87, 2.22], [3, 738030, 'Bayern Munich', 'Hertha', 5, 0, 'H', 2, 0, 'H', 17, 8, 6, 3, 4, 5, 8, 3, 0, 0, 0, 0, 1.18, 7.96, 14.67], [3, 738024, 'Bayern Munich', 'FC Koln', 3, 2, 'H', 0, 0, 'D', 18, 10, 6, 3, 14, 10, 3, 4, 1, 1, 0, 0, 1.13, 9.35, 19.76]]
[[3, 738051, 'Bayern Munich', 'Bochum', 1, 0, 0, 0, 3, 0, 1.07, 13.19, 31.99], [0, 738045, 'Bochum', 'Hertha', 1, 3, 'A', 0, 2, 'A', 19, 5, 5, 4, 9, 17, 7, 2, 0, 3, 0, 0, 2.78, 3.35, 2.58], [3, 738030, 'FC Koln', 'Bochum', 2, 1, 'H', 0, 0, 'D', 21, 10, 5, 1, 11, 21, 5, 5, 2, 1, 0, 0, 1.88, 3.73, 4.11], [3, 738023, 'Bochum', 'Mainz', 2, 0, 'H', 1, 0, 'H', 9, 16, 5, 3, 15, 19, 2, 7, 4, 3, 0, 0, 2.66, 3.51, 2.6]]


In [205]:
longTest = homeList[0] + homeList[1] + homeList[2] + homeList[3]
longTest = longTest + awayList[1] + awayList[2] + awayList[3]
print(longTest)

[3, 738051, 'Bayern Munich', 'Bochum', 1, 0, 0, 0, 3, 0, 1.07, 13.19, 31.99, 0, 738044, 'RB Leipzig', 'Bayern Munich', 1, 4, 'A', 0, 1, 'A', 10, 18, 4, 6, 9, 13, 2, 6, 0, 0, 0, 0, 2.99, 3.87, 2.22, 3, 738030, 'Bayern Munich', 'Hertha', 5, 0, 'H', 2, 0, 'H', 17, 8, 6, 3, 4, 5, 8, 3, 0, 0, 0, 0, 1.18, 7.96, 14.67, 3, 738024, 'Bayern Munich', 'FC Koln', 3, 2, 'H', 0, 0, 'D', 18, 10, 6, 3, 14, 10, 3, 4, 1, 1, 0, 0, 1.13, 9.35, 19.76, 0, 738045, 'Bochum', 'Hertha', 1, 3, 'A', 0, 2, 'A', 19, 5, 5, 4, 9, 17, 7, 2, 0, 3, 0, 0, 2.78, 3.35, 2.58, 3, 738030, 'FC Koln', 'Bochum', 2, 1, 'H', 0, 0, 'D', 21, 10, 5, 1, 11, 21, 5, 5, 2, 1, 0, 0, 1.88, 3.73, 4.11, 3, 738023, 'Bochum', 'Mainz', 2, 0, 'H', 1, 0, 'H', 9, 16, 5, 3, 15, 19, 2, 7, 4, 3, 0, 0, 2.66, 3.51, 2.6]


In [222]:
masterList = list()
for index, row in dfJoined.iterrows():
    homeString = row["HomeTeam"]
    awayString = row["AwayTeam"]
    ordNumber = row["OrdDate"]

#    print(homeString)
#    print(type(homeString))
#    print(awayString)
#    print(type(awayString))
#    print(ordNumber)
#    print(type(ordNumber))
    # Return an empty list if there is no dict entry matching ordNumber
    homeList = resultDict[homeString].get(ordNumber, list())
    awayList = resultDict[awayString].get(ordNumber, list())
    if (len(homeList) > 0) & (len(awayList) > 0):
        resultList = homeList[0] + homeList[1] + homeList[2] + homeList[3]
        resultList = resultList + awayList[1] + awayList[2] + awayList[3]
        masterList.append(resultList)

print (len(masterList))

879


In [223]:
print(masterList[0])

[1, 738044, 'Freiburg', 'FC Koln', 0, 0, 0, 0, 0, 0, 2.14, 3.7, 3.25, 0, 738030, 'Stuttgart', 'Freiburg', 2, 3, 'A', 2, 3, 'A', 17, 14, 7, 7, 10, 14, 8, 5, 2, 1, 0, 0, 2.23, 3.53, 3.22, 3, 738023, 'Freiburg', 'Dortmund', 2, 1, 'H', 1, 0, 'H', 8, 19, 4, 2, 14, 14, 0, 10, 1, 4, 0, 0, 5.5, 4.59, 1.55, 1, 738016, 'Bielefeld', 'Freiburg', 0, 0, 'D', 0, 0, 'D', 10, 14, 4, 6, 13, 10, 4, 9, 3, 4, 0, 0, 2.92, 3.27, 2.52, 3, 738030, 'FC Koln', 'Bochum', 2, 1, 'H', 0, 0, 'D', 21, 10, 5, 1, 11, 21, 5, 5, 2, 1, 0, 0, 1.88, 3.73, 4.11, 3, 738024, 'Bayern Munich', 'FC Koln', 3, 2, 'H', 0, 0, 'D', 18, 10, 6, 3, 14, 10, 3, 4, 1, 1, 0, 0, 1.13, 9.35, 19.76, 3, 738017, 'FC Koln', 'Hertha', 3, 1, 'H', 1, 1, 'D', 15, 11, 5, 5, 18, 14, 4, 5, 4, 2, 0, 0, 2.57, 3.34, 2.8]


The last significant step is preparing the column array for converting the list of lists into a dataframe. Notepad++ was my assistant here; I gave it string to replace all _H1 with _H2, then _H3, and finally all _H with _A. The result worked the first time I tried it, which is always nice.

In [226]:
columnArray = ['y', 'ordToday', 'homeToday', 'awayToday', 'MHOrd', 'MAOrd', 'MHLoc', 'MALoc', 'MHRes', 'MARes', 'AvgH', 'AvgD', 'AvgA',
               'y_H1', 'ord_H1', 'home_H1', 'away_H1', 'FTHG_H1', 'FTAG_H1', 'FTR_H1', 'HTHG_H1', 'HTAG_H1', 'HTR_H1', 'HS_H1', 'AS_H1',
               'HST_H1', 'AST_H1', 'HF_H1', 'AF_H1', 'HC_H1', 'AC_H1', 'HY_H1', 'AY_H1', 'HR_H1', 'AR_H1', 'AvgH_H1', 'AvgD_H1', 'AvgA_H1',
               'y_H2', 'ord_H2', 'home_H2', 'away_H2', 'FTHG_H2', 'FTAG_H2', 'FTR_H2', 'HTHG_H2', 'HTAG_H2', 'HTR_H2', 'HS_H2', 'AS_H2',
               'HST_H2', 'AST_H2', 'HF_H2', 'AF_H2', 'HC_H2', 'AC_H2', 'HY_H2', 'AY_H2', 'HR_H2', 'AR_H2', 'AvgH_H2', 'AvgD_H2', 'AvgA_H2',
               'y_H3', 'ord_H3', 'home_H3', 'away_H3', 'FTHG_H3', 'FTAG_H3', 'FTR_H3', 'HTHG_H3', 'HTAG_H3', 'HTR_H3', 'HS_H3', 'AS_H3',
               'HST_H3', 'AST_H3', 'HF_H3', 'AF_H3', 'HC_H3', 'AC_H3', 'HY_H3', 'AY_H3', 'HR_H3', 'AR_H3', 'AvgH_H3', 'AvgD_H3', 'AvgA_H3',
               'y_A1', 'ord_A1', 'home_A1', 'away_A1', 'FTHG_A1', 'FTAG_A1', 'FTR_A1', 'HTHG_A1', 'HTAG_A1', 'HTR_A1', 'HS_A1', 'AS_A1',
               'HST_A1', 'AST_A1', 'HF_A1', 'AF_A1', 'HC_A1', 'AC_A1', 'HY_A1', 'AY_A1', 'HR_A1', 'AR_A1', 'AvgH_A1', 'AvgD_A1', 'AvgA_A1',
               'y_A2', 'ord_A2', 'home_A2', 'away_A2', 'FTHG_A2', 'FTAG_A2', 'FTR_A2', 'HTHG_A2', 'HTAG_A2', 'HTR_A2', 'HS_A2', 'AS_A2',
               'HST_A2', 'AST_A2', 'HF_A2', 'AF_A2', 'HC_A2', 'AC_A2', 'HY_A2', 'AY_A2', 'HR_A2', 'AR_A2', 'AvgH_A2', 'AvgD_A2', 'AvgA_A2',
               'y_A3', 'ord_A3', 'home_A3', 'away_A3', 'FTHG_A3', 'FTAG_A3', 'FTR_A3', 'HTHG_A3', 'HTAG_A3', 'HTR_A3', 'HS_A3', 'AS_A3',
               'HST_A3', 'AST_A3', 'HF_A3', 'AF_A3', 'HC_A3', 'AC_A3', 'HY_A3', 'AY_A3', 'HR_A3', 'AR_A3', 'AvgH_A3', 'AvgD_A3', 'AvgA_A3']

In [227]:
dfMaster = pd.DataFrame(masterList, columns = columnArray)
dfMaster.head()

,y,ordToday,homeToday,awayToday,MHOrd,MAOrd,MHLoc,MALoc,MHRes,MARes,...,AF_A3,HC_A3,AC_A3,HY_A3,AY_A3,HR_A3,AR_A3,AvgH_A3,AvgD_A3,AvgA_A3
0,1,738044,Freiburg,FC Koln,0,0,0,0,0,0,...,14,4,5,4,2,0,0,2.57,3.34,2.80
1,0,738044,Greuther Furth,Wolfsburg,0,0,0,0,0,0,...,7,5,2,0,2,0,1,1.57,4.37,5.63
2,0,738044,Hoffenheim,Mainz,0,0,0,0,0,0,...,8,5,11,1,2,0,0,8.07,5.44,1.35
3,0,738044,Leverkusen,Dortmund,0,0,0,0,0,0,...,14,7,3,0,0,0,0,1.44,5.17,6.49
4,1,738044,Union Berlin,Augsburg,0,0,0,0,0,0,...,9,6,5,1,2,0,0,3.19,3.68,2.17


In [228]:
print(dfMaster.shape)

(879, 163)


Now that we have the merged master datafile for each match, I'll save it off in pickle and CSV format. (I'll include both in this project's GitHub folder.) Since my home laptop doesn't have a decent GPU, I'll move to my Google Colab account for the last phase of this project.

In [230]:
dfMaster.to_pickle('data/buffalo/master.pkl')

In [233]:
dfMaster.to_csv('data/buffalo/master.csv')